# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/mtp-thesis

/content/drive/MyDrive/mtp-thesis


In [ ]:
ls

# Loading Data

In [4]:
DATA_FILE = '/content/drive/MyDrive/mtp-thesis/data/X_train_YG7NZSq.csv'

In [5]:
RESULTS_FOLDER = '/content/drive/MyDrive/mtp-thesis/results'

In [6]:
import pandas as pd
import numpy as np

In [7]:
_data = pd.read_csv(DATA_FILE)
_datam = np.matrix(_data)

In [ ]:
_data.T.describe()

In [ ]:
_datam.shape

# Data Handler Class and Utils Function

In [10]:
import numpy as np

def gram_schmidt_algorithm(A):
    """
    O(d^2k)
    """
    for i in range(A.shape[0]):
        q = A[i, :]
        for j in range(i):
            q = q - np.dot(A[j,:], A[i,:]) * A[j,:]
        q = q / np.sqrt(np.dot(q, q))
        A[i,:] = q
    return A

def cayley_transformation(A):
    """
    A is (d,k) matrix
    T = 
    O()
    """
    I = np.eye(A.shape[0])
    Q = np.matmul(np.linalg.inv(I+A),(I-A))
    return Q

def random_matrix(shape, limits_gap=200, center=0.5):
    rmat = np.random.random(shape)
    while np.linalg.matrix_rank(rmat) < min(shape[0],shape[1]):
        rmat = np.random.random(shape)
    return limits_gap*(rmat-center)

class Data_Handler:
    def __init__(self, n, k, d, T, S, R):
        """
        R is a n x (T + S) numpy array
        d is an integer that represents the number of time lags
        d <= k
        """
        self.n = n
        self.k = k
        self.d = d
        self.T = T
        self.S = S
        self.R = R
        assert(d<=k)
        assert(R.shape[0]==n)
        assert(R.shape[1]==(T+S))
        self._compute_optimizers()
        pass

    def get_string(self):
        return f"{self.n}-{self.k}-{self.d}-{self.T}-{self.S}"

    def _compute_optimizers(self):
        self.sum_RtdTRtd_dT_altD = np.zeros((self.d, self.d))  # d x d
        self.sum_RtdTRt_dT_altN = np.zeros((self.d, 1))  # d x 1
        self.list_RtdTRtd = []
        self.list_RtdTRt = []
        self.list_norm_Rt = []
        for t in range(self.d):
            r_t = self.R[:, t].reshape(self.n, 1)  # n x 1
            self.list_RtdTRtd += [None]
            self.list_RtdTRt += [None]
            self.list_norm_Rt += [np.linalg.norm(r_t, 2)]
        for t in range(self.d, self.T+self.S):
            r_t = self.R[:, t].reshape(self.n, 1)  # n x 1
            r_td = self.R[:, t-self.d:t]  # n x d
            self.list_RtdTRtd += [np.matmul(r_td.T, r_td)]
            self.list_RtdTRt += [np.matmul(r_td.T, r_t)]
            self.list_norm_Rt += [np.linalg.norm(r_t, 2)]
        for t in range(self.d, self.T):
            self.sum_RtdTRtd_dT_altD += self.list_RtdTRtd[t]
            self.sum_RtdTRt_dT_altN += self.list_RtdTRt[t]

    def compute_reward(self, beta, A):
        Abeta = np.matmul(A, beta)  # d x 1
        reward_sum = 0
        for t in range(self.T, self.T + self.S):
            reward_sum += np.matmul(self.list_RtdTRt[t].T, Abeta) / (self.list_norm_Rt[t] * np.sqrt(np.matmul(Abeta.T, np.matmul(self.list_RtdTRtd[t], Abeta))))
        reward = reward_sum / self.S
        return reward

    def compute_loss(self, beta, A):
        Abeta = np.matmul(A, beta)  # d x 1
        loss_sum = 0
        for t in range(self.d, self.T):
            loss_sum += self.list_norm_Rt[t]
            loss_sum += -2*np.matmul(self.list_RtdTRt[t].T, Abeta)
            loss_sum += np.matmul(Abeta.T, np.matmul(self.list_RtdTRtd[t], Abeta))
        loss = loss_sum / (2*(self.T - self.d + 1))
        return loss

    def compute_loss_gradient_beta(self, beta, A):
        beta_grad = np.matmul(A.T, np.matmul(self.sum_RtdTRtd_dT_altD, np.matmul(A, beta))) - np.matmul(A.T, self.sum_RtdTRt_dT_altN)
        beta_grad = beta_grad / (self.T - self.d + 1)
        return beta_grad

    def compute_loss_gradient_A(self, beta, A):
        A_grad = np.matmul(self.sum_RtdTRtd_dT_altD, np.matmul(A, beta)) - self.sum_RtdTRt_dT_altN
        A_grad = np.matmul(A_grad, beta.T)
        A_grad = A_grad / (self.T - self.d + 1)
        return A_grad

    def compute_loss_gradient_beta_A(self, beta, A):
        temp = np.matmul(self.sum_RtdTRtd_dT_altD, np.matmul(A, beta))
        beta_grad = np.matmul(A.T, temp) - np.matmul(A.T, self.sum_RtdTRt_dT_altN)
        beta_grad = beta_grad / (self.T - self.d + 1)
        A_grad = temp - self.sum_RtdTRt_dT_altN
        A_grad = np.matmul(A_grad, beta.T)
        A_grad = A_grad / (self.T - self.d + 1)
        return beta_grad, A_grad

    def compute_reward_gradient_beta(self, beta, A):
        T1 = np.zeros((self.d,1))
        T2 = np.zeros((self.d,self.d))
        Abeta = np.matmul(A, beta)  # d x 1
        for t in range(self.T, self.T + self.S):
            Ftbeta_norm = float(np.sqrt(np.matmul(Abeta.T,np.matmul(self.list_RtdTRtd[t],Abeta))))
            RtFtbeta_scalar = float(np.matmul(self.list_RtdTRt[t].T, Abeta))
            T1 += self.list_RtdTRt[t]/(Ftbeta_norm * self.list_norm_Rt[t])
            T2 += (RtFtbeta_scalar/(self.list_norm_Rt[t]*Ftbeta_norm**3)  * self.list_RtdTRtd[t])
        grad_beta = np.matmul(A.T, T1) - np.matmul(A.T, np.matmul(T2,Abeta))
        grad_beta =  grad_beta / self.S
        return grad_beta

    def compute_reward_gradient_A(self, beta, A):
        T1 = np.zeros((self.d,1))
        T2 = np.zeros((self.d,self.d))
        Abeta = np.matmul(A, beta)  # d x 1
        for t in range(self.T, self.T + self.S):
            Ftbeta_norm = float(np.sqrt(np.matmul(Abeta.T,np.matmul(self.list_RtdTRtd[t],Abeta))))
            RtFtbeta_scalar = float(np.matmul(self.list_RtdTRt[t].T, Abeta))
            T1 += self.list_RtdTRt[t]/(Ftbeta_norm * self.list_norm_Rt[t])
            T2 += (RtFtbeta_scalar/(self.list_norm_Rt[t]*Ftbeta_norm**3)  * self.list_RtdTRtd[t])
        grad_A = np.matmul(T1, beta.T) - np.matmul(np.matmul(T2, Abeta), beta.T)
        grad_A =  grad_A / self.S
        return grad_A
    
    def compute_reward_gradient_beta_A(self, beta, A):
        T1 = np.zeros((self.d,1))
        T2 = np.zeros((self.d,self.d))
        Abeta = np.matmul(A, beta)  # d x 1
        for t in range(self.T, self.T + self.S):
            Ftbeta_norm = float(np.sqrt(np.matmul(Abeta.T,np.matmul(self.list_RtdTRtd[t],Abeta))))
            RtFtbeta_scalar = float(np.matmul(self.list_RtdTRt[t].T, Abeta))
            T1 += self.list_RtdTRt[t]/(Ftbeta_norm * self.list_norm_Rt[t])
            T2 += (RtFtbeta_scalar/(self.list_norm_Rt[t]*Ftbeta_norm**3)  * self.list_RtdTRtd[t])
        grad_beta = np.matmul(A.T, T1) - np.matmul(A.T, np.matmul(T2,Abeta))
        grad_beta =  grad_beta / self.S
        grad_A = np.matmul(T1, beta.T) - np.matmul(np.matmul(T2, Abeta), beta.T)
        grad_A =  grad_A / self.S
        return grad_beta, grad_A
    
    def compute_orthogonal_condition_normF(self, A):
        return np.sum((np.matmul(A, A.T) - np.eye(A.shape[0]))**2)

    def compute_orthogonal_condition_normF_gradient_A(self, A):
        return -4.0*np.matmul(np.eye(A.shape[0]) - np.matmul(A, A.T), A)

    def linear_l2_regression(self, A):
        """
        O(d^2k+k^3) = O(k^3)
        """
        beta = np.matmul(np.matmul(A.T,self.sum_RtdTRtd_dT_altD),A)
        try:
            beta = np.linalg.inv(beta)
            beta = np.matmul(beta, np.matmul(A.T,self.sum_RtdTRt_dT_altN))
            return beta
        except:
            print(f"faced error")
            print(f"going for least squared method to solve it")
            gamma = np.matmul(A.T,self.sum_RtdTRt_dT_altN)
            beta = np.linalg.lstsq(beta,gamma)
            return beta





# Logger 

In [ ]:
!pip install gif

In [ ]:
!pip install --ignore-installed Pillow==9.0.0

In [13]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import gif

class Logger:
    def __init__(self, log_name: str, data_handler:Data_Handler):
        self.log_name = log_name
        self.list_R = []
        self.list_L = []
        self.list_O = []
        self.EA_RLO_logs = dict()
        self.EA_archive_RLO = dict()

        self.data_handler = data_handler
        pass

    def clear(self):
        self.list_R = []
        self.list_L = []
        self.list_O = []
        self.EA_RLO_logs = dict()
        self.EA_archive_RLO = dict()
        pass

    def log_RLO(self, R, L, O):
        self.list_L += [float(L)]
        self.list_R += [float(R)]
        self.list_O += [float(O)]
        pass
    
    def log_EA_RLO(self,gen,R,L,O):
        R = float(R)
        L = float(L)
        O = float(O)
        if gen in self.EA_RLO_logs.keys():
            self.EA_RLO_logs[gen][0].append(R)
            self.EA_RLO_logs[gen][1].append(L)
            self.EA_RLO_logs[gen][2].append(O)
        else:
            self.EA_RLO_logs[gen] =  [[R],[L],[O]]
        pass  
        
    def log_EA_Population(self, gen, P): 
        R = [float(P[i].R) for i in range(len(P))]
        L = [float(P[i].L) for i in range(len(P))]
        O = [float(self.data_handler.compute_orthogonal_condition_normF(P[i].A)) for i in range(len(P))]
        self.EA_RLO_logs[gen] = [R,L,O]

    def log_archive(self,gen,Arch):
        R = [float(Arch[i].R) for i in range(len(Arch))]
        L = [float(Arch[i].L) for i in range(len(Arch))]
        O = [float(self.data_handler.compute_orthogonal_condition_normF(Arch[i].A)) for i in range(len(Arch))]
        self.EA_archive_RLO[gen] = [R,L,O]
        pass
    
    def dump(self,folder):
        if len(self.list_L) > 0:
            df = pd.DataFrame()
            df['R'] = self.list_R
            df['L'] = self.list_L
            df['O'] = self.list_O
            fname = "RLO_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            df.to_csv(fpath, index = False)
            print(f" {self.log_name} : dumped ROL")
        
        if len(self.EA_RLO_logs) > 0:
            df = pd.DataFrame()
            for k,v in self.EA_RLO_logs.items():
                df[f"{k}_R"] = v[0]
                df[f"{k}_L"] = v[1]
                df[f"{k}_O"] = v[2]
            fname = "EARLO_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            df.to_csv(fpath, index = False)
            print(f" {self.log_name} : dumped EARLO")

        if len(self.EA_archive_RLO) > 0:
            l = []
            for k,v in self.EA_archive_RLO.items():
                l.append((f"{k}_R", pd.Series(v[0])))
                l.append((f"{k}_L", pd.Series(v[1])))
                l.append((f"{k}_O", pd.Series(v[2])))
            df = pd.DataFrame(dict(l))
            fname = "EAarchiveRLO_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            df.to_csv(fpath, index = False)
            print(f" {self.log_name} : dumped EAarchiveROL")
        pass

    def load(self,rlo_file=None, earlo_file=None, eaarchiverl_file=None):
        if rlo_file is not None:
            df = pd.read_csv(rlo_file)
            self.list_R = list(df['R'])
            self.list_L = list(df['L'])
            self.list_O = list(df['O'])
        else:
            self.list_R = []
            self.list_L = []
            self.list_O = []
        
        self.EA_RLO_logs = dict()
        if earlo_file is not None:
            df = pd.read_csv(earlo_file)
            for k in df.columns:
                g = int(k.split('_')[0])
                if g in self.EA_RLO_logs.keys():
                    continue
                self.EA_RLO_logs[g] = [list(df[g+"_R"]),list(df[g+"_L"]),list(df[g+"_O"])]
        
        self.EA_archive_RLO = dict()
        if eaarchiverl_file is not None:
            df = pd.read_csv(eaarchiverl_file)
            for k in df.columns:
                g = int(k.split('_')[0])
                if g in self.EA_archive_RLO.keys():
                    continue
                self.EA_archive_RLO[g] = [list(df[g+"_R"]),list(df[g+"_L"]),list(df[g+"_O"])]
        pass


    def plot_scatter_LR(self, x_scale="linear", y_scale="linear", save=False, file_name="fig.png"):
        sns.set_style("darkgrid")
        sns.set_palette("bright")
        x_label = "Reward"
        y_label = "Loss"
        ax = sns.scatterplot(x=self.list_R, y=self.list_L)
        ax.set(xscale=x_scale, yscale=y_scale, xlabel=x_label, ylabel=y_label)
        if save:
            plt.savefig(file_name)
        plt.show()

    def dist_plot(self, save=False, filename='distplot.png'):
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
        sns.set_style('whitegrid')
        sns.distplot(self.list_R, label='Reward', ax=ax1)
        ax1.set(title='Distribution of Rewards')
        sns.distplot(self.list_L, label='Loss', ax=ax2)
        ax2.set(title='Distribution of Loss')
        ax1.legend()
        ax2.legend()
        if save:
            plt.savefig(filename)
        plt.show()
    
    def get_gif_from_RL_log_dict(self, duration=200,style='seaborn',xscale='log',yscale='log',dpi=300,lam_margins=1.2, set_boundaries = False, use_archive_instead=False):
        plt.style.use(style)
        gif.options.matplotlib["dpi"] = dpi

        def get_min_max_limits(lam):
            R_max = -float('inf')
            R_min = float('inf')
            L_max = -float('inf')
            L_min = float('inf')
            if use_archive_instead:
                for k,v in self.EA_archive_RLO.items():
                    R_max = max(np.max(v[0]),R_max)
                    R_min = min(np.min(v[0]),R_min)
                    L_max = max(np.max(v[1]),L_max)
                    L_min = min(np.min(v[1]),L_min)
            else:
                for k,v in self.EA_RLO_logs.items():
                    R_max = max(np.max(v[0]),R_max)
                    R_min = min(np.min(v[0]),R_min)
                    L_max = max(np.max(v[1]),L_max)
                    L_min = min(np.min(v[1]),L_min)
            return 0.5*np.asarray([  (1+lam)*R_min+(1-lam)*R_max, (1-lam)*R_min+(1+lam)*R_max, (1+lam)*L_min+(1-lam)*L_max, (1-lam)*L_min+(1+lam)*L_max])
            
        limits = get_min_max_limits(lam_margins)
        @gif.frame
        def plot(key,value):
            # sns.set(style="whitegrid")
            fig, ax = plt.subplots()
            ax.scatter(x=value[0], y=value[1], color='red')
            if set_boundaries:
                ax.set_xlim([limits[0], limits[1]])
                ax.set_ylim([limits[2], limits[3]])
            ax.set_xlabel('Reward')
            ax.set_ylabel('Loss')
            ax.set_yscale(yscale)
            ax.set_xscale(xscale)
            ax.set_title(f"{self.log_name} Generation : {key}")
        frames = []
        if use_archive_instead:
            frames += [ plot(k,v) for k,v in self.EA_archive_RLO.items()]
        else:
            frames += [ plot(k,v) for k,v in self.EA_RLO_logs.items()]
        filename = f"{self.log_name}" + ".gif"
        gif.save(frames, filename, duration=duration)
        pass

    def get_tragectory_gif(self,duration=200,style='seaborn',xscale='log',yscale='log',dpi=300,lam_margins=1.2,set_boundaries = False):
        plt.style.use(style)
        gif.options.matplotlib["dpi"] = dpi

        def get_min_max_limits(lam):
            R_max = -float('inf')
            R_min = float('inf')
            L_max = -float('inf')
            L_min = float('inf')
            for k,v in self.EA_RLO_logs.items():
                R_max = max(np.max(v[0]),R_max)
                R_min = min(np.min(v[0]),R_min)
                L_max = max(np.max(v[1]),L_max)
                L_min = min(np.min(v[1]),L_min)
            return 0.5*np.asarray([  (1+lam)*R_min+(1-lam)*R_max, (1-lam)*R_min+(1+lam)*R_max, (1+lam)*L_min+(1-lam)*L_max, (1-lam)*L_min+(1+lam)*L_max])
        
        limits = get_min_max_limits(lam_margins)
        @gif.frame
        def plot(gen_i):
            fig, ax = plt.subplots()
            for point in range(len(self.EA_RLO_logs[gen_i][0])):
                point_Rs = [self.EA_RLO_logs[gg][0][point] for gg in range(gen_i+1)]
                points_Ls = [self.EA_RLO_logs[gg][1][point] for gg in range(gen_i+1)]
                # ax.plot(point_Rs, points_Ls, label=f'Point {point+1}')
                ax.plot(point_Rs, points_Ls)
                if set_boundaries:
                    ax.set_xlim([limits[0], limits[1]])
                    ax.set_ylim([limits[2], limits[3]])

            # Set plot properties
            ax.set_xlabel('Reward')
            ax.set_ylabel('Loss')
            ax.set_title(f"Trajectories of Points: {gen_i}")
            ax.set_yscale(yscale)
            ax.set_xscale(xscale)

        gen_max = len(self.EA_RLO_logs)
        frames = [ plot(i) for i in range(gen_max)]
        filename = f"{self.log_name}-tragectory" + ".gif"
        gif.save(frames, filename, duration=duration)
        pass

    def show_tragetctory_plot(self,style='seaborn',xscale='log',yscale='log'):
        plt.style.use(style)
        fig, ax = plt.subplots()
        gen_max = len(self.EA_RLO_logs)
        # Plot each point's trajectory as a line
        for point in range(len(self.EA_RLO_logs[0][0])):
            point_Rs = [self.EA_RLO_logs[gg][0][point] for gg in range(gen_max)]
            points_Ls = [self.EA_RLO_logs[gg][1][point] for gg in range(gen_max)]
            ax.plot(point_Rs, points_Ls, label=f'Point {point+1}')

        # Set plot properties
        ax.set_xlabel('Reward')
        ax.set_ylabel('Loss')
        ax.set_title('Trajectories of Points')
        ax.set_yscale(yscale)
        ax.set_xscale(xscale)
        # ax.legend()
        plt.show()

    def get_grad_gif(self,duration=200,style='seaborn',xscale='log',yscale='log',dpi=300,lam_margins=1.2,set_boundaries = False):
        plt.style.use(style)
        gif.options.matplotlib["dpi"] = dpi

        def get_min_max_limits(lam):
            R_max = -float('inf')
            R_min = float('inf')
            L_max = -float('inf')
            L_min = float('inf')
            for k,v in self.EA_RLO_logs.items():
                R_max = max(np.max(v[0]),R_max)
                R_min = min(np.min(v[0]),R_min)
                L_max = max(np.max(v[1]),L_max)
                L_min = min(np.min(v[1]),L_min)
            return 0.5*np.asarray([  (1+lam)*R_min+(1-lam)*R_max, (1-lam)*R_min+(1+lam)*R_max, (1+lam)*L_min+(1-lam)*L_max, (1-lam)*L_min+(1+lam)*L_max])
        
        limits = get_min_max_limits(lam_margins)
        @gif.frame
        def plot(gen_i):
            fig, ax = plt.subplots()
            for point in range(len(self.EA_RLO_logs[gen_i][0])):
                point_Rs = [self.EA_RLO_logs[gg][0][point] for gg in range(gen_i-1,gen_i+1)]
                points_Ls = [self.EA_RLO_logs[gg][1][point] for gg in range(gen_i-1,gen_i+1)]
                # ax.plot(point_Rs, points_Ls, label=f'Point {point+1}')
                ax.plot(point_Rs, points_Ls)
                if set_boundaries:
                    ax.set_xlim([limits[0], limits[1]])
                    ax.set_ylim([limits[2], limits[3]])

            # Set plot properties
            ax.set_xlabel('Reward')
            ax.set_ylabel('Loss')
            ax.set_title(f"Trajectories of Points: {gen_i}")
            ax.set_yscale(yscale)
            ax.set_xscale(xscale)

        gen_max = len(self.EA_RLO_logs)
        frames = [ plot(i) for i in range(1,gen_max)]
        filename = f"{self.log_name}-directiongrad" + ".gif"
        gif.save(frames, filename, duration=duration)
        pass


    def pareto_front_plot(self):
        """
        using all the solution ever generated in a method
        """
        pass

    def get_3d_graph(self):
        pass

    def compute_core_statistics(folder, self, last_from_list__=200, beyond_gen=10):
        if len(self.list_L) > 0:
            df = pd.DataFrame()
            df['R'] = self.list_R[-last_from_list__:]
            df['L'] = list(-np.log(self.list_L[-last_from_list__:]))
            df['O'] = list(-np.log(self.list_O[-last_from_list__:]))
            dfd = df.describe()
            print(dfd)

            fname = f"CORE_STATS_{last_from_list__}_RLO_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            dfd.to_csv(fpath, index = False)
            print(f" {fname} : dumped ROL")
        
        if len(self.EA_RLO_logs) > 0:
            from_dict = self.EA_RLO_logs

            df = pd.DataFrame()
            df['R'] = []
            df['L'] = []
            df['O'] = []
            df['gen'] = []
            for k,v in from_dict.items():
                if k < beyond_gen:
                    continue
                df1 = pd.DataFrame()
                df1["R"] = v[0]
                df1["L"] = list(-np.log(v[1]))
                df1["O"] = list(-np.log(v[2]))
                df1['gen'] = len(v[0])*[k]
                df = pd.concat([df, df1], axis=0)
            dfd = df.describe()
            grouped_df = df.groupby('gen').mean()
            print(dfd)
            print(grouped_df)

            fname = "CORE_STATS_EARLO_{beyond_gen}_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            dfd.to_csv(fpath, index = False)
            print(f" {fname} : dumped EARLO")

            fname = "gen_change_EARLO_{beyond_gen}_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            dfd.to_csv(fpath, index = False)
            print(f" {fname} : dumped change gen EARLO")

        if len(self.EA_archive_RLO) > 0:
            from_dict = self.EA_archive_RLO

            df = pd.DataFrame()
            df['R'] = []
            df['L'] = []
            df['O'] = []
            df['gen'] = []
            for k,v in from_dict.items():
                if k < beyond_gen:
                    continue
                df1 = pd.DataFrame()
                df1["R"] = v[0]
                df1["L"] = list(-np.log(v[1]))
                df1["O"] = list(-np.log(v[2]))
                df1['gen'] = len(v[0])*[k]
                df = pd.concat([df, df1], axis=0)
            dfd = df.describe()
            grouped_df = df.groupby('gen').mean()
            print(dfd)
            print(grouped_df)

            fname = "CORE_STATS_EAarchiveROL_{beyond_gen}_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            dfd.to_csv(fpath, index = False)
            print(f" {fname} : dumped EAarchiveROL")

            fname = "gen_change_EAarchiveROL_{beyond_gen}_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            dfd.to_csv(fpath, index = False)
            print(f" {fname} : dumped change gen EAarchiveROL")
        pass

# EA Helpers

In [14]:
import numpy as np
from tqdm import tqdm

class Solution:
    error_eps = 1e-6
    def __init__(self):
        self.beta = None
        self.A = None

        self.R = None
        self.L = None
        self.obj = [-self.R, self.L]

        self.crowding_distance = None
        self.domination_count = None

        self.best_beta = None
        self.best_A = None
        self.best_R = None
        self.best_L = None
        self.best_obj = [-self.best_R, self.best_L]

        self.vel_A = None
        self.vel_beta = None

        pass

    def __init__(self, beta, A,vel_beta =None, vel_A=None):
        self.beta = beta
        self.A = A
        self.vel_beta = vel_beta
        self.vel_A = vel_A
        pass

    def __str__(self):
        return f"{self.R},{self.L}"

    def assign_RL(self, R, L):
        self.R = R
        self.L = L
        self.obj = [-self.R, self.L]
        pass

    def first_set_RL_NSGA(self, R, L):
        self.R = R
        self.L = L
        self.obj = [-self.R, self.L]
        pass

    def first_set_RL_MOPSO(self, R, L):
        self.R = R
        self.L = L
        self.obj = [-self.R, self.L]

        self.best_beta = self.beta
        self.best_A = self.A
        self.best_R = R
        self.best_L = L
        self.best_obj = [-self.best_R, self.best_L]
        pass

    def set_best_pos_obj_MOPSO(self, _beta, _A,_R, _L ):
        self.best_beta = _beta
        self.best_A = _A
        self.best_R = _R
        self.best_L = _L
        self.best_obj = [-self.best_R, self.best_L]
        pass
    
    def update_best_pos_obj_MOPSO(self,pos_candi):
        if pos_candi.dominates(self):
            self.set_best_pos_obj_MOPSO(pos_candi.beta,pos_candi.A,pos_candi.R,pos_candi.L)
            return True
        elif not self.dominates(pos_candi):
            if np.random.random() > 0.5:
                self.set_best_pos_obj_MOPSO(pos_candi.beta,pos_candi.A,pos_candi.R,pos_candi.L)
                return True
        return False

    def dominates(self,other):
        diff = False
        for j in range(2):
            if self.obj[j] > other.obj[j] + Solution.error_eps:
                return False
            elif self.obj[j] < other.obj[j] - Solution.error_eps:
                diff = True
        return diff
    
def nondominated_subset(P):
    n_p = len(P)
    for p in P:
        p.domination_count = 0
    cand_ids = set(range(n_p))
    for i in range(n_p):
        if i not in cand_ids:
            continue
        dom_ids = set()
        for j in cand_ids:
            if j==i:
                continue
            if P[i].dominates(P[j]):
                dom_ids.add(j)
        cand_ids.difference_update(dom_ids)
    non_dom = [P[i] for i in cand_ids]
    return non_dom

def non_dominated_sorting(P, serialize = False):
    n_p = len(P)
    for p in P:
        p.domination_count = 0
    for i in range(len(P)):
        for j in range(len(P)):
            if i!=j:
                if P[i].dominates(P[j]):
                    P[j].domination_count += 1
    P = sorted(P, key=lambda p:p.domination_count)

    F_list = [[P[0]]]
    last_dom_c, i =P[0].domination_count,1
    while i < n_p:
        if P[i].domination_count == last_dom_c:
            F_list[-1].append(P[i])
        else:
            F_list.append([P[i]])
            last_dom_c = P[i].domination_count
        i+=1
    
    F_list_return = []
    if serialize:
        for F_i in F_list:
            F_list_return += F_i
    else:
        F_list_return = F_list
    
    return F_list_return

def compute_crowding_distance(P, is_nondominant=False, serialize=False, to_sort=False):
    F_list = []
    if not is_nondominant:
        F_list = non_dominated_sorting(P)
    else:
        F_list = [P]
    
    for F_i in F_list:
        for p in F_i:
            p.crowding_distance = 0
        for j in range(2):
            F_i = sorted(F_i,key= lambda p: p.obj[j])
            for k in range(1, len(F_i)-1):
                F_i[k].crowding_distance += abs(F_i[k +1].obj[j]-F_i[k-1].obj[j])
            F_i[0].crowding_distance = np.inf
            F_i[len(F_i)-1].crowding_distance = np.inf
    
    if is_nondominant:
        if to_sort:
            F_list[0] = sorted(F_list[0], key=lambda p:p.crowding_distance)
        return F_list[0]
    else:
        if serialize:
            F_list_return = []
            for F_i in F_list:
                F_list_return+=F_i
            return F_list_return
        else:
            return F_list

# NSGA-II

## Code

In [15]:
import numpy as np
import copy 
from tqdm import tqdm

class NSGA2:

    def __init__(self, data_handler: Data_Handler):
        self.n = data_handler.n
        self.k = data_handler.k
        self.d = data_handler.d
        self.T = data_handler.T
        self.S = data_handler.S
        self.data_handler = data_handler
        pass
    
    def get_random_solution_population(self,N):
        P = []
        for i in range(N):
            A = random_matrix((self.d,self.k))
            A = gram_schmidt_algorithm(A)
            beta = self.data_handler.linear_l2_regression(A)
            P.append(Solution(beta,A))
        return P

    def get_binary_tournament_selection(self,sample_size,N,p_t):
        i = 0
        p_tresh = p_t
        sample = set()
        while len(sample) < sample_size and i < N:
            if np.random.random() < p_tresh:
                sample.add(i)
            p_tresh = p_tresh*(1-p_t)
            i+=1

        while len(sample) < sample_size:
            sample.add(np.random.randint(0,N))

        return sample
    
    def get_offspring_population(self, N, P, p_c, p_m, p_t, mutation_step_size,crossover_step_size, use_reg_for_crossover):
        C = []
        for i in range(N//2):
            sids = list(self.get_binary_tournament_selection(2,N,p_t))
            S1, S2 = P[sids[0]], P[sids[1]]

            if np.random.random() < p_c:
                S1,S2 = self.crossover(S1,S2,crossover_step_size, use_reg_for_crossover)
            if np.random.random() < p_m:
                S1 = self.mutate(S1,mutation_step_size)
                S2 = self.mutate(S2,mutation_step_size)
            C += [S1,S2]
        return C

    def crossover(self,S1:Solution,S2:Solution,crossover_step_size,use_reg_for_crossover):
        C1, C2 = copy.deepcopy(S1), copy.deepcopy(S2)
        deltaA = crossover_step_size*(C2.A -C1.A)
        T_1 =  0.5*(np.matmul(deltaA.T,C1.A) - np.matmul(C1.A.T,deltaA))
        T_2 =  0.5*(-np.matmul(deltaA.T,C2.A) + np.matmul(C2.A.T,deltaA))
        Q1 = cayley_transformation(T_1)
        Q2 = cayley_transformation(T_2)
        C1.A = np.matmul(C1.A,Q1)
        C2.A = np.matmul(C2.A,Q2)

        deltabeta = crossover_step_size*(C2.beta -C1.beta)
        if use_reg_for_crossover:
            C1.beta = self.data_handler.linear_l2_regression(C1.A)
            C2.beta = self.data_handler.linear_l2_regression(C2.A)
        else:
            C1.beta = C1.beta + deltabeta
            C2.beta = C2.beta - deltabeta
        return C1,C2

    def mutate(self,S:Solution,mutation_step_size):
        Sm = copy.deepcopy(S)
        deltabeta = random_matrix((self.k,1))
        deltaA = random_matrix((self.d,self.k))
        T_ =  0.5*mutation_step_size*(np.matmul(deltaA.T,Sm.A) - np.matmul(Sm.A.T,deltaA))
        Q =  cayley_transformation(T_)
        Sm.A = np.matmul(Sm.A,Q)
        Sm.beta = Sm.beta+mutation_step_size*deltabeta
        return Sm

    def run(self, N, G, p_c, p_m,p_t, mutation_step_size,crossover_step_size,use_reg_for_crossover:bool,logger:Logger=None):
        """
        p_t ~ 1/n add a factor of n to algorithm i.e. 1/p_t so keep p_t constant
        """
        assert(N%2==0)
        P = self.get_random_solution_population(N)
        for p in P:
            _R = self.data_handler.compute_reward(p.beta,p.A)
            _L = self.data_handler.compute_loss(p.beta,p.A)
            p.first_set_RL_NSGA(_R,_L)
        P = non_dominated_sorting(P,True)
        if logger is not None:
            logger.log_EA_Population(0,P)
        for gen in tqdm(range(G), desc="Generation of NSGA2"):
            C = self.get_offspring_population(N, P, p_c, p_m,p_t,mutation_step_size,crossover_step_size, use_reg_for_crossover)
            for c in C:
                _R = self.data_handler.compute_reward(c.beta,c.A)
                _L = self.data_handler.compute_loss(c.beta,c.A)
                c.first_set_RL_NSGA(_R,_L)
            F_list = non_dominated_sorting(P+C)
            _P = []
            n_gen = len(F_list)
            i = 0
            while i < n_gen and len(_P)+len(F_list[i]) < N:
                _P += F_list[i]
                i += 1
            if len(_P) < N and i < n_gen:
                F_list[i] = compute_crowding_distance(F_list[i],is_nondominant=True,serialize=False,to_sort=True)
                _P += F_list[i][-(N-len(_P)):]
            if logger is not None:
                logger.log_EA_Population(gen+1,_P)
            P = _P
        return P
    
    def get_string(self, N, G, p_c, p_m, p_t, mutation_step_size,crossover_step_size, use_reg_for_crossover):
        return f"NSGAII[{1*use_reg_for_crossover},{N},{G},{int(1000*p_c)},{int(1000*p_m)},{int(1000*p_t)},{int(1000*mutation_step_size)},{int(1000*crossover_step_size)}]"

## Experiment

In [16]:
n = 50
k = 21
d = 20
T = 600
assert(T<_datam.shape[1])
S = _datam.shape[1]-T

dh = Data_Handler(n,k,d,T,S,_datam)

#### Without Regression

In [25]:
pop_size = 50
generations = 100
prob_crossover = 1
prob_mutation = 0.2
prob_tournament = 0.1
mutation_setp_size = 1e-2
crossover_setp_size = 1e-2
use_reg_for_crossover = False

In [22]:
nsga2 = NSGA2(dh)
logger_name = nsga2.get_string(pop_size,
              generations,
              prob_crossover,
              prob_mutation,
              prob_tournament,
              mutation_setp_size,
              crossover_setp_size,
              use_reg_for_crossover) + '-' + dh.get_string()
logger_nsga2 = Logger(logger_name,dh)

In [23]:
P = nsga2.run(pop_size,
              generations,
              prob_crossover,
              prob_mutation,
              prob_tournament,
              mutation_setp_size,
              crossover_setp_size,
              use_reg_for_crossover,
              logger_nsga2)

Generation of NSGA2: 100%|██████████| 50/50 [01:20<00:00,  1.61s/it]


In [ ]:
logger_nsga2.dump(RESULTS_FOLDER)

In [ ]:
for pop_size_i in [100, 200, 500] :
  nsga2 = NSGA2(dh)
  logger_name = nsga2.get_string(pop_size_i,
                generations,
                prob_crossover,
                prob_mutation,
                prob_tournament,
                mutation_setp_size,
                crossover_setp_size,
                use_reg_for_crossover) + '-' + dh.get_string()
  logger_nsga2 = Logger(logger_name,dh)
  P = nsga2.run(pop_size_i,
                generations,
                prob_crossover,
                prob_mutation,
                prob_tournament,
                mutation_setp_size,
                crossover_setp_size,
                use_reg_for_crossover,
                logger_nsga2)
  logger_nsga2.dump(RESULTS_FOLDER)

In [ ]:
logger_nsga2.get_gif_from_RL_log_dict(duration=200,xscale='linear',yscale='log',set_boundaries=False)

#### With Regression

In [17]:
pop_size = 100
generations = 100
prob_crossover = 1
prob_mutation = 0.2
prob_tournament = 0.1
mutation_setp_size = 1e-2
crossover_setp_size = 1e-2
use_reg_for_crossover = True

In [28]:
nsga2 = NSGA2(dh)
logger_name = nsga2.get_string(pop_size,
              generations,
              prob_crossover,
              prob_mutation,
              prob_tournament,
              mutation_setp_size,
              crossover_setp_size,
              use_reg_for_crossover) + '-' + dh.get_string()
logger_nsga2 = Logger(logger_name,dh)

In [ ]:
P = nsga2.run(pop_size,
              generations,
              prob_crossover,
              prob_mutation,
              prob_tournament,
              mutation_setp_size,
              crossover_setp_size,
              use_reg_for_crossover,
              logger_nsga2)

In [ ]:
for pop_size_i in [100, 200, 500] :
  nsga2 = NSGA2(dh)
  logger_name = nsga2.get_string(pop_size_i,
                generations,
                prob_crossover,
                prob_mutation,
                prob_tournament,
                mutation_setp_size,
                crossover_setp_size,
                use_reg_for_crossover) + '-' + dh.get_string()
  logger_nsga2 = Logger(logger_name,dh)
  P = nsga2.run(pop_size_i,
                generations,
                prob_crossover,
                prob_mutation,
                prob_tournament,
                mutation_setp_size,
                crossover_setp_size,
                use_reg_for_crossover,
                logger_nsga2)
  logger_nsga2.dump(RESULTS_FOLDER)

In [ ]:
logger_nsga2.get_gif_from_RL_log_dict(duration=200,xscale='linear',yscale='log',set_boundaries=False)

In [ ]:
logger_nsga2.EA_RLO_logs[0] == logger_nsga2.EA_RLO_logs[19]

# MOPSO

## Code

In [13]:
import numpy as np
from tqdm import tqdm
import copy

class MOPSO:
    def __init__(self, data_handler: Data_Handler):
        self.n = data_handler.n
        self.k = data_handler.k
        self.d = data_handler.d
        self.T = data_handler.T
        self.S = data_handler.S
        self.data_handler = data_handler
        pass

    def initialize_population_and_velocity(self,N):
        P = []
        for i in range(N):
            A = random_matrix((self.d,self.k))
            A = gram_schmidt_algorithm(A)
            beta = self.data_handler.linear_l2_regression(A)
            vel_beta = self.data_handler.compute_reward_gradient_beta(beta, A)
            vel_A = self.data_handler.compute_reward_gradient_A(beta, A)
            P+=[Solution(beta,A,vel_beta,vel_A)]
        return P

    def update_velocity(self, p:Solution, best_global:Solution,w,c_1,c_2):
        r1, r2 = np.random.random(), np.random.random()
        p.vel_beta = w*best_global.vel_beta+c_1*r1*(p.best_beta-p.beta)+c_2*r2*(best_global.best_beta-p.beta)
        p.vel_A = w*best_global.vel_A+c_1*r1*(p.best_A-p.A)+c_2*r2*(best_global.best_A-p.A)
        pass

    def update_position(self, p:Solution, eps):
        T_ =  0.5*eps*(np.matmul(p.vel_A.T, p.A) - np.matmul(p.A.T,p.vel_A))
        Q = cayley_transformation(T_)
        p.A = np.matmul(p.A,Q)
        p.beta = p.beta + eps*p.vel_beta
        pass

    def mutate(self, Sm:Solution, mutation_step_size, vel_mutation_setp_size):
        deltabeta = random_matrix((self.k,1))
        deltaA = random_matrix((self.d,self.k))
        T_ = 0.5*mutation_step_size*(np.matmul(deltaA.T, Sm.A) - np.matmul(Sm.A.T,deltaA))
        Q =  cayley_transformation(T_)
        Sm.A = np.matmul(Sm.A,Q)
        Sm.beta = Sm.beta+mutation_step_size*deltabeta
        Sm.vel_beta = Sm.vel_beta + vel_mutation_setp_size*random_matrix((self.k,1))
        Sm.vel_A = Sm.vel_A + vel_mutation_setp_size*random_matrix((self.d,self.k))
        pass

    def best_global_from(self, archive, p_gb_nr):
        """
        assumes archive is sorted wrt crowding distance 
        """
        if np.random.random() < p_gb_nr:
            # return the one with maximum crowding distance with prob p_gb_nr
            return archive[-1] 
        else:
            rid = np.random.randint(0,len(archive))
            return archive[rid]

    def non_dominated_merge(self,M,archive, P):
        F = nondominated_subset(P)
        archive = nondominated_subset(F+archive)
        if len(archive) > M:
            archive = compute_crowding_distance(archive, is_nondominant=True, to_sort=True)
            archive = archive[-M:]
        return archive

    def run(self,N, M, T_max,w,c_1,c_2,eps,p_m,mutate_step_size,vel_mutation_setp_size, p_gb_nr, logger:Logger=None):
        Archive = []
        P = self.initialize_population_and_velocity(N)
        for p in P:
            _R = self.data_handler.compute_reward(p.beta,p.A)
            _L = self.data_handler.compute_loss(p.beta,p.A)
            p.first_set_RL_MOPSO(_R,_L)
        Archive = nondominated_subset(P)
        Archive = compute_crowding_distance(Archive,is_nondominant=True, to_sort=True)
        if logger is not None:
            logger.log_EA_Population(0,P)
            logger.log_archive(0,Archive)
        for t in tqdm(range(T_max), desc="Time loop in MOPSO"):
            best_global = self.best_global_from(Archive,p_gb_nr)
            # for i in tqdm(range(N), desc="Position update loop in MOPSO"):
            for i in range(N):
                self.update_velocity(P[i], best_global, w,c_1,c_2)
                self.update_position(P[i],eps)
                if np.random.random() < p_m:
                    self.mutate(P[i],mutate_step_size, vel_mutation_setp_size)
                _R = self.data_handler.compute_reward(P[i].beta,P[i].A)
                _L = self.data_handler.compute_loss(P[i].beta,P[i].A)
                P[i].assign_RL(_R,_L)
                _temp_sol = Solution(p.beta, p.A)
                _temp_sol.assign_RL(_R,_L)
                p.update_best_pos_obj_MOPSO(_temp_sol)
            Archive = self.non_dominated_merge(M,Archive,P)
            Archive = compute_crowding_distance(Archive,is_nondominant=True, to_sort=True)
            if logger is not None:
                logger.log_EA_Population(t+1,P)
                logger.log_archive(t+1,Archive)
        return Archive
    
    def get_string(self,N, M, T_max,w,c_1,c_2,eps,p_m,mutate_step_size,vel_mutation_setp_size, p_gb_nr):
        return f"MOPSO[{N},{M},{T_max},{int(1000*w)},{int(1000*c_1)},{int(1000*c_2)},{int(1000*eps)},{int(1000*p_m)},{int(1000*mutate_step_size)},{int(1000*vel_mutation_setp_size)},{int(1000*p_gb_nr)}]"

## Experiment

In [ ]:
n = 50
k = 21
d = 20
T = 600
assert(T<_datam.shape[1])
S = _datam.shape[1]-T

dh = Data_Handler(n,k,d,T,S,_datam)

In [ ]:
pop_size = 20
archive_size = 1000
generation = 100
velocity_decay_weight = 0.90
c_1 = 0.5
c_2 = 0.2
position_update_eps = 6*1e-3
prob_mutation = 0.1
pos_mutation_setp_size = 1e-3
vel_mutation_setp_size = 1e-3
prob_non_random_global_best = 0.95

In [ ]:
mopso = MOPSO(dh)
logger_name = mopso.get_string(pop_size,
              archive_size,
              generation,
              velocity_decay_weight,
              c_1,
              c_2,
              position_update_eps,
              prob_mutation,
              pos_mutation_setp_size,
              vel_mutation_setp_size,
              prob_non_random_global_best) + '-' + dh.get_string()
logger_mopso = Logger(logger_name,dh)
P = mopso.run(pop_size,
              archive_size,
              generation,
              velocity_decay_weight,
              c_1,
              c_2,
              position_update_eps,
              prob_mutation,
              pos_mutation_setp_size,
              vel_mutation_setp_size,
              prob_non_random_global_best,
              logger=logger_mopso)


In [ ]:
len(P)

53

In [ ]:
logger_mopso.dump(RESULTS_FOLDER)

In [ ]:
for pop_size_i in [100, 200, 500] :
  mopso = MOPSO(dh)
  logger_name = mopso.get_string(pop_size_i,
                archive_size,
                generation,
                velocity_decay_weight,
                c_1,
                c_2,
                position_update_eps,
                prob_mutation,
                pos_mutation_setp_size,
                vel_mutation_setp_size,
                prob_non_random_global_best) + '-' + dh.get_string()
  logger_mopso = Logger(logger_name,dh)
  P = mopso.run(pop_size_i,
                archive_size,
                generation,
                velocity_decay_weight,
                c_1,
                c_2,
                position_update_eps,
                prob_mutation,
                pos_mutation_setp_size,
                vel_mutation_setp_size,
                prob_non_random_global_best,
                logger=logger_mopso)
  logger_mopso.dump(RESULTS_FOLDER)

In [ ]:
logger_mopso.show_tragetctory_plot(xscale='linear')

In [ ]:
logger_mopso.get_tragectory_gif(xscale='linear')

In [ ]:
logger_mopso.get_grad_gif(xscale='linear')

In [ ]:
logger_mopso.get_gif_from_RL_log_dict(xscale='linear',style='ggplot',use_archive_instead=True)